# 競馬スクレイピング

目的：

競馬データについてスクレイピングを行う

手順：

1.レースIDのリストを作成
race_id_list

2.レースIDから、レース情報についてスクレイピングを行いデータフレームを作成
race_infos

3.レースIDから、レース結果についてスクレイピングを行いデータフレームを作成
race_results

In [1]:
#ライブラリインポート

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from tqdm.notebook import tqdm
import itertools
import requests

## レースIDを作成

In [2]:
#レースIDのリスト作成
#URL構造　"https://db.netkeiba.com/race/"　+　race_id
#race_id構造　年,開催場所,回数,何日目か,レース

race_id_list = []
for year in range(2020,2021):
    #(2008,2021)
    for place in range(1, 11):
        #(1, 11)
        for kai in range(1, 7):
            #(1,7)
            for day in range(1, 13):
                #(1,13)
                for r in range(1, 13):
                    #(1,13)
                    race_id = str(year).zfill(4) + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                    race_id_list.append(race_id)


## レース情報についてスクレイピングする関数を定義

In [3]:
def scrape_race_info(race_id_list):
    race_infos = {}
    for race_id in race_id_list:
        try:
            #レース結果ページから、レースの情報について収集し、整形する
            html = urlopen('https://db.netkeiba.com/race/' + race_id)
            bs = BeautifulSoup(html.read(),'html.parser')

            #レース情報についてスクレイピング
            #[レース名、距離、天気、馬場、馬場状態、日付]
            texts = (
                        bs.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                        + bs.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                        )
            race_name = bs.find("div", attrs={"class": "data_intro"}).find_all("h1")[0].text
            #infoに正規表現を使ってtextsを取得
            info = re.findall(r"\w+", texts)

            #整形
            info_dict = {}
            for text in info:
                info_dict["race_name"] = race_name
                if "m" in text:
                    info_dict["course_len"] = int(re.findall(r"\d+", text)[0])
                if "m" in text:
                    info_dict["rotation"] = str(re.findall(r"[右,左]", text)[0])
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    info_dict["weather"] = text
                if text in ["芝", "ダート"]:
                    info_dict["race_type"] = text
                if text in ["良", "稍重", "重", "不良"]:
                    info_dict["ground_state"] = text
                if "障" in text:
                    info_dict["race_type"] = "障害"
                if "年" in text:
                    info_dict["date"] = text
        
            race_infos[race_id] = info_dict
            #time.sleep(1)
        except IndexError:
            continue
        except AttributeError:
            continue
        except Exception as e:
            print(e)
            break
        except:
            break
    return race_infos

### 関数実行

In [4]:
#関数実行
race_infos=scrape_race_info(race_id_list)

#DataFrame型に変更
race_infos = pd.DataFrame(race_infos).T
race_infos = race_infos.rename_axis('race_id', axis = 'columns')

race_infos


race_id,race_name,course_len,rotation,weather,race_type,ground_state,date
202001010101,2歳未勝利,1800,右,曇,芝,良,2020年7月25日
202001010102,3歳未勝利,1700,右,曇,ダート,良,2020年7月25日
202001010103,3歳未勝利,1500,右,曇,芝,良,2020年7月25日
202001010104,3歳未勝利,1700,右,曇,ダート,良,2020年7月25日
202001010105,2歳新馬,1200,右,晴,芝,良,2020年7月25日
...,...,...,...,...,...,...,...
202010020808,3歳以上1勝クラス,1000,右,雨,ダート,稍重,2020年9月6日
202010020809,別府特別,1200,右,雨,芝,重,2020年9月6日
202010020810,西日本スポーツ杯,1800,右,雨,芝,重,2020年9月6日
202010020811,第40回小倉2歳ステークス(G3),1200,右,雨,芝,重,2020年9月6日


## レース結果についてスクレイピングをする関数を定義

In [5]:
#レース結果スクレイピング
def get_results(tmp_df, race_id):
    url = "https://db.netkeiba.com/race/" + race_id

    try:
        
        page = requests.get(url)
        
        soup = BeautifulSoup(page.content, 'lxml')
        
        table1 = soup.find('table', class_='race_table_01 nk_tb_common')
        
        headers = ['race_id']
        for i in table1.find_all('th'):
            title = i.text
            headers.append(title)

        df = pd.DataFrame(columns = headers)

        for j in table1.find_all('tr')[1:]:
            row_data = j.find_all('td')
            row = [i.text.rstrip() for i in row_data]
            row = [[race_id], row]
            row = list(itertools.chain.from_iterable(row))
            
            length = len(df)
            df.loc[length] = row

        df['馬名'] = df['馬名'].apply(lambda x: x.split('\n')[1])
        df['騎手'] = df['騎手'].apply(lambda x: x.split('\n')[1])

        # 着順に数字以外の文字列が含まれているものを取り除く
        df = df[~(df["着順"].astype(str).str.contains("\D"))]
        df["着順"] = df["着順"].astype(int)
        
        # 性齢を性と年齢に分ける
        df["性"] = df["性齢"].map(lambda x: str(x)[0])
        df["年齢"] = df["性齢"].map(lambda x: str(x)[1:]).astype(int)

        # 馬体重を体重と体重変化に分ける
        df["体重"] = df["馬体重"].str.split("(", expand=True)[0].astype(int)
        df["体重変化"] = df["馬体重"].str.split("(", expand=True)[1].str[:-1].astype(int)

        # データをint, floatに変換
        df["単勝"] = df["単勝"].astype(float)

        # 不要な列を削除
        df = df.drop(['ﾀｲﾑ指数', '通過', '着差', '性齢', '馬体重', '調教ﾀｲﾑ', '厩舎ｺﾒﾝﾄ', '備考', '調教師',
        '馬主', '賞金(万円)'], axis=1)

        race_results = pd.concat([tmp_df, df])
        return race_results
    
    except:
        return tmp_df

In [6]:
##　空のデータフレーム作成
headers = ['race_id', '着順', '枠番', '馬番', '馬名', '斤量', '騎手', 'タイム', '上り', '単勝', '人気',
       '性', '年齢', '体重', '体重変化']
tmp_df = pd.DataFrame(columns = headers)

In [7]:
## 指定のレース結果全て１つのデータフレームにする
for race in tqdm(race_id_list):
    race_results = get_results(tmp_df, race)
    tmp_df = race_results

  0%|          | 0/8640 [00:00<?, ?it/s]

In [8]:
race_results

,race_id,着順,枠番,馬番,馬名,斤量,騎手,タイム,上り,単勝,人気,性,年齢,体重,体重変化
0,202001010101,1,6,6,ウインルーア,54,横山武史,1:49.7,35.6,16.0,3,牝,2,438,4
1,202001010101,2,2,2,アークライト,54,ルメール,1:50.0,35.8,1.9,2,牡,2,510,0
2,202001010101,3,3,3,ギャラントウォリア,54,池添謙一,1:50.1,36.2,1.8,1,牡,2,482,-6
3,202001010101,4,1,1,ジュンブーケ,52,亀田温心,1:50.5,36.7,22.2,4,牝,2,442,0
4,202001010101,5,4,4,キタノマンゲツ,54,藤岡康太,1:51.0,36.6,55.7,5,牡,2,426,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,202010020812,11,1,1,キスラー,57,武豊,1:45.5,37.7,6.5,4,牡,4,476,-2
11,202010020812,12,8,14,ポンペイワーム,57,浜中俊,1:45.6,38.0,19.8,8,牡,4,522,-8
12,202010020812,13,8,15,アルマトップエンド,57,国分恭介,1:45.9,37.3,75.0,14,牡,5,510,0
13,202010020812,14,2,3,ノイーヴァ,55,太宰啓介,1:46.7,37.8,46.1,10,牝,4,488,-12


In [10]:
#CSV形式で保存
race_infos.to_csv("race_infos_2020.csv",encoding="shift_jis",index = False)
race_results.to_csv("race_results_2020.csv",encoding="shift_jis", index = False)